# Window Operations Exercise Solution

### Exercise

In [1]:
import findspark
# TODO: your path will likely not have 'jilg' in it. Change it to reflect your path.
findspark.init('/home/jilg/Downloads/spark-3.3.2-bin-hadoop3')

In [2]:
import os
import sys

from pyspark.sql import Row, SparkSession
from pyspark.sql.streaming import DataStreamWriter, DataStreamReader
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import window

In [3]:
spark = SparkSession.builder.appName("WindowedCount").master("local[*]").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [4]:
# TODO: Define a windowDuration and slideDuration
windowDuration = '50 seconds'
slideDuration = '30 seconds'

In [5]:
# TODO: Set up the `lines` readStream to take in data from a socket stream, AND include the timestamp
lines = spark.readStream.format("socket").option("host", "localhost")\
         .option("port", 6669).option('includeTimestamp', 'true').load()

In [6]:
# Splitting the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"),lines.timestamp)

In [7]:
# TODO: Add a watermark to the data, using the timestamp
words = words.withWatermark("timestamp", "5 seconds")

In [8]:
# TODO: Write out the windowed wordcounts using groupBy(), window(), and count().
windowedCounts = words.groupBy(window(words.timestamp, windowDuration, slideDuration),words.word).count()

In [ ]:
query = windowedCounts.writeStream.outputMode("complete")\
                   .option("numRows", "100000")\
                   .option("truncate", "false")\
                   .format("console")\
                   .start()

query.awaitTermination()